In [1]:
import pandas as pd
import csv
import os, os.path
from sklearn.preprocessing import LabelEncoder
import sklearn as sk  
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm  
from sklearn.linear_model import LogisticRegression 
from sklearn.cluster import KMeans
from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.cluster import SpectralClustering
from sklearn.neural_network import MLPClassifier

# import numpy as np
# from sklearn import preprocessing
# import matplotlib.pyplot as plt 
# import seaborn as sns
# from numpy import array


In [2]:
#creating my dataframe
mydata=pd.DataFrame({"Db": [], "Hz": []})
labels=pd.DataFrame({"Male": [], "Feeling": [], "Person":[]})

In [3]:
students=['\\mario','\\giuliano','\\cristina'] #directory of records of each student
for i in students:
    
    cwd = os.getcwd() #current working directory
    sd = cwd + str(i) #student directory
    lab = pd.read_csv(str(sd)+'\\labels.csv', sep=';') #read the labels file
    labels=pd.concat([labels,lab], ignore_index=True)
    
    for k in range(1,len(os.listdir(sd))): #runs the cycle with respect to number of records for each student
        
        read = pd.read_csv(str(sd)+'\\sl Registrazione '+str(k)+'.csv', sep = ',') #read data  
        
        Db=read["DecibelSource"].dropna()  #clean from NAs
        Hz=read["PitchSensor"].dropna()    
        
        Db=list(Db)#make them to list to easier work later
        Hz=list(Hz)
        
        mydata=mydata.append({'Db' : Db , 'Hz' : Hz} , ignore_index=True)

    
result=pd.concat([mydata,labels], sort=False, axis=1) #merge labels and data

In [4]:
result.head()

,Db,Hz,Male,Feeling,Person
0,"[51.509708077318315, 49.206463639207875, 53.09...","[83.5404052734375, 83.5404052734375, 83.540405...",1.0,angry,other
1,"[55.10859961906438, 51.864688959738814, 51.652...","[291.0975477430556, 331.6312255859375, 331.631...",0.0,happy,other
2,"[29.52738094783193, 27.660100580824576, 25.288...","[268.5400390625, 230.9423828125, 230.942382812...",1.0,steady,other
3,"[33.119446531464924, 28.733594908973828, 33.64...","[419.1162109375, 419.1162109375, 419.116210937...",1.0,sad,other
4,"[19.45318383545405, 18.038502361992837, 16.372...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 339.233398...",0.0,angry,other


# Cleaning

In [5]:
#see if there are problem with classes
print(set(result.Person))
print(set(result.Feeling))

{'other ', 'other', 'giuliano', 'mario', 'cristina'}
{'happy ', 'serious', 'angry', 'happy', 'sad ', 'sad', 'steady'}


In [6]:
#fix them
for i in range(len(result)):
    if result['Feeling'][i]=='happy ':
        result['Feeling'][i]='happy'
    if result['Feeling'][i]=='serious':
        result['Feeling'][i]='steady'
    if result['Person'][i]=='other ':
        result['Person'][i]='other'
    if result['Feeling'][i]=='sad ':
        result['Feeling'][i]='sad'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_lau

#### Categorical encoding

In [7]:
##transform every cathegorical var into numbers
le=LabelEncoder()

cat=['Feeling','Person']

for i in cat:
    result[i]=le.fit_transform(result[i].astype(str))

In [8]:
result.head()

,Db,Hz,Male,Feeling,Person
0,"[51.509708077318315, 49.206463639207875, 53.09...","[83.5404052734375, 83.5404052734375, 83.540405...",1.0,0,3
1,"[55.10859961906438, 51.864688959738814, 51.652...","[291.0975477430556, 331.6312255859375, 331.631...",0.0,1,3
2,"[29.52738094783193, 27.660100580824576, 25.288...","[268.5400390625, 230.9423828125, 230.942382812...",1.0,3,3
3,"[33.119446531464924, 28.733594908973828, 33.64...","[419.1162109375, 419.1162109375, 419.116210937...",1.0,2,3
4,"[19.45318383545405, 18.038502361992837, 16.372...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 339.233398...",0.0,0,3


In [9]:
#are the feeling balanced?
print(str(len(result.loc[result["Feeling"]==0]))+' angry records')
print(str(len(result.loc[result["Feeling"]==1]))+' happy records')
print(str(len(result.loc[result["Feeling"]==2]))+' sad records')
print(str(len(result.loc[result["Feeling"]==3]))+' steady records')
print('')
print(str(len(result.loc[result["Male"]==0]))+' female records')
print(str(len(result.loc[result["Male"]==1]))+' male records')

69 angry records
61 happy records
59 sad records
76 steady records

162 female records
103 male records


In [10]:
#evaluate mininum len
mini=100000000000000000 
for i in range(len(result)):
    now=len(result["Hz"][i]) #see lenght of first HZ vector
#     print(str(i)+ " " +str(now)) #print number of vector and his lenght
    if now<mini: #see if is the shortest vector
        mini=now #if shortest, update minimum value
# mini

In [66]:
##SOLO UNA PROVA!!! PROVARE AD USARE FRAMMENTI DI REGISTRAZIONE MOLTO PIU PICCOLI
# mini=20

In [67]:
#same len for all

for i in range(len(result)):
    result["Db"][i]=result["Db"][i][0:mini]

for i in range(len(result)):
    result["Hz"][i]=result["Hz"][i][0:mini]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [12]:
#selecting only male and female to study their properties
male=result.loc[result["Male"]==1]
male=male.reset_index(drop=True)  #setting new indexes for the new DF
female=result.loc[result["Male"]==0]
female=female.reset_index(drop=True)

In [68]:
##let's see how the means differs
somma=0
for i in range(len(male)):
    somma+=mean(male["Db"][i])
print("mean for DB in male is "+str(round(somma/(len(male)+1),2)))

somma=0
for i in range(len(male)):
    somma+=mean(male["Hz"][i])
print("mean for HZ in male is " + str(round(somma/(len(male)+1),2)))

##now for female..
somma=0
for i in range(len(female)):
    somma+=mean(female["Db"][i])
print("mean for DB in female is "+str(round(somma/(len(female)+1),2)))

somma=0
for i in range(len(female)):
    somma+=mean(female["Hz"][i])
print("mean for HZ in female is " +str(round(somma/(len(female)+1),2)))

mean for DB in male is 45.04
mean for HZ in male is 308.47
mean for DB in female is 44.38
mean for HZ in female is 272.48


# Modelling

In [69]:
#Creating our X matrix
cols=['Db','Hz']#selecting only hz and db data
mat=[] #empty matrix
for c in cols:
    for k in range(mini):
        l=[] #empty array
        for i in range(len(result)):
            l.append(result[c][i][k]) #filling the array making it the column we are going to use
        mat.append(l) #append our column to the matrix
X=pd.DataFrame(mat) #make it a pandas df
X=X.T #transpose it
X=pd.concat([X,result.Person],axis=1)#add person column

In [70]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,Person
0,51.509708,49.206464,53.098951,52.469692,53.167057,52.176484,48.654212,53.256975,52.688264,56.267311,...,407.724609,407.724609,208.979492,208.979492,208.979492,208.979492,211.816406,211.816406,211.816406,3
1,55.108600,51.864689,51.652674,47.638640,49.127333,48.002193,53.917669,54.711934,54.078948,51.856219,...,49.003855,49.003855,49.003855,93.824463,93.824463,93.824463,93.824463,93.824463,79.567732,3
2,29.527381,27.660101,25.288264,20.859257,22.952716,26.111523,39.505439,35.678947,34.919236,32.231507,...,0.000000,462.571010,462.571010,462.571010,343.046177,343.046177,343.046177,232.177734,232.177734,3
3,33.119447,28.733595,33.640379,45.195911,46.322613,38.580593,42.709851,39.776593,30.642523,24.077781,...,257.285156,257.285156,130.504842,130.504842,130.504842,130.504842,130.866648,130.866648,130.866648,3
4,19.453184,18.038502,16.372480,15.836151,23.020714,23.356387,21.816119,22.132527,21.352715,19.915726,...,419.000244,419.000244,419.000244,419.000244,0.000000,0.000000,0.000000,0.000000,0.000000,3


first 123 columns are the decibel records, while other 123 are the pitch record. we just want to add columns with summary of them in order to keep the relationship between values

In [71]:
#Adding mean, max, min and SD columns, for DB and for HZ
X['dbMean']=X.iloc[:,0:mini].mean(axis=1)
X['dbMax']=X.iloc[:,0:mini].max(axis=1)
X['dbMin']=X.iloc[:,0:mini].min(axis=1)
X['dbSd']=X.iloc[:,0:mini].std(axis=1)

X['hzMean']=X.iloc[:,mini:mini*2].mean(axis=1)
X['hzMax']=X.iloc[:,mini:mini*2].max(axis=1)
X['hzMin']=X.iloc[:,mini:mini*2].min(axis=1)
X['hzSd']=X.iloc[:,mini:mini*2].std(axis=1)

In [72]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,39,Person,dbMean,dbMax,dbMin,dbSd,hzMean,hzMax,hzMin,hzSd
0,51.509708,49.206464,53.098951,52.469692,53.167057,52.176484,48.654212,53.256975,52.688264,56.267311,...,211.816406,3,50.819216,56.267311,45.391763,2.927105,271.797780,558.574219,83.540405,180.847925
1,55.108600,51.864689,51.652674,47.638640,49.127333,48.002193,53.917669,54.711934,54.078948,51.856219,...,79.567732,3,53.610306,57.178711,47.638640,2.794113,147.807673,331.631226,34.603928,119.882361
2,29.527381,27.660101,25.288264,20.859257,22.952716,26.111523,39.505439,35.678947,34.919236,32.231507,...,232.177734,3,32.278234,41.733744,20.859257,5.754009,232.400559,462.571010,0.000000,139.556540
3,33.119447,28.733595,33.640379,45.195911,46.322613,38.580593,42.709851,39.776593,30.642523,24.077781,...,130.866648,3,37.301217,48.091666,24.077781,6.500940,240.903927,419.116211,130.504842,99.482297
4,19.453184,18.038502,16.372480,15.836151,23.020714,23.356387,21.816119,22.132527,21.352715,19.915726,...,0.000000,3,24.759123,49.115809,14.676970,9.009366,151.646729,419.000244,0.000000,192.302860


In [73]:
#setting our y 
y=result.Male

#splitting train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Random forest
## on Gender

In [74]:
RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)  
RF.fit(X_train, y_train)  
RF.predict(X_test)

array([0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [75]:
RF.score(X_test,y_test)

0.6704545454545454

# Support vector machine 
## on Gender

In [76]:
SVM = svm.LinearSVC()  
SVM.fit(X_train, y_train)  
SVM.predict(X_test) 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


array([0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
       0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0.,
       1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0.,
       0., 0., 0.])

In [77]:
SVM.score(X_test,y_test)

0.5681818181818182

# Logistic regression
## on Gender

In [78]:
LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)  
LR.predict(X_test)  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


array([0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1.,
       0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0.,
       1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0.,
       0., 0., 0.])

In [79]:
LR.score(X_test,y_test)

0.5227272727272727

# Neural Network
## on Gender

In [80]:
NN = MLPClassifier(solver='lbfgs', alpha=0.1, hidden_layer_sizes=(5, 2), random_state=1)  
NN.fit(X_train, y_train)  
NN.predict(X_test)

array([0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 0.])

In [81]:
NN.score(X_test, y_test)

0.6818181818181818

# MODELS ON FEELING

In [82]:
y=result.Feeling 


#splitting train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Support vector machine 
## on Feeling

In [83]:
SVM = svm.SVC(decision_function_shape="ovo")  
SVM.fit(X_train, y_train)  
SVM.predict(X_test) 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [84]:
SVM.score(X_test,y_test)

0.26136363636363635

# Logistic regression
## on Feeling

In [85]:
LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)  
LR.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


array([2, 1, 0, 3, 2, 3, 2, 1, 2, 1, 3, 0, 1, 3, 3, 3, 3, 3, 1, 1, 0, 1,
       3, 2, 2, 1, 1, 1, 2, 0, 0, 3, 1, 0, 0, 0, 2, 0, 2, 3, 3, 2, 2, 3,
       0, 1, 1, 0, 0, 3, 2, 3, 0, 2, 0, 1, 3, 3, 3, 0, 3, 2, 2, 3, 0, 3,
       0, 1, 1, 2, 2, 0, 2, 3, 0, 3, 1, 2, 0, 2, 3, 1, 3, 3, 2, 2, 3, 2])

In [86]:
LR.score(X_test,y_test)

0.42045454545454547

# Random forest
## on Feeling

In [87]:
RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)  
RF.fit(X_train, y_train)  
RF.predict(X_test)

array([0, 3, 3, 3, 3, 3, 2, 3, 0, 3, 2, 0, 3, 3, 3, 3, 0, 3, 3, 2, 0, 3,
       0, 3, 0, 3, 3, 3, 3, 0, 2, 3, 3, 3, 0, 0, 2, 0, 0, 3, 1, 2, 0, 3,
       3, 1, 2, 0, 3, 3, 3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 3, 0, 1, 0, 0, 3,
       0, 3, 3, 2, 0, 0, 2, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3])

In [88]:
RF.score(X_test,y_test)

0.3068181818181818

# Neural Network
## on Feeling

In [89]:
NN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)  
NN.fit(X_train, y_train)  
NN.predict(X_test)

array([0, 0, 3, 0, 0, 3, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0,
       0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 3,
       3, 3, 3, 0, 0, 3, 3, 0, 0, 0, 0, 3, 3, 0, 3, 0, 3, 0, 0, 0, 0, 0,
       0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 3, 3, 3, 0, 3, 0, 0, 0, 0])

In [90]:
NN.score(X_test, y_test)

0.20454545454545456

# Clustering

In [91]:
##set the kmean model
kmeans = KMeans(init='k-means++', n_init=10, n_clusters=2, random_state=0).fit(list(result["Db"]))

In [92]:
##predict the gender from Db for each line in the matrix
count=0 #to count how many times it's right
for i in range(len(result)):
    pred=kmeans.predict(list([result["Db"][i]])) #say in which cluster is the datapoint db, i
    true=result["Male"][i] #the true Male value of line i 
    
#     print(str(pred)+" "+str(true))
    if pred==true: #if he get the real value
        count+=1
# print(str(count)+"/"+str(len(result)))     #print how many times it predict  
print(count/len(result))

0.5924528301886792


In [ ]:
"""#let's now encode the cathegorical variables
toAdd=pd.get_dummies(result['Person']).reset_index(drop=True)
result=pd.concat([result,toAdd], axis=1, sort=False)

toAdd=pd.get_dummies(result['Feeling']).reset_index(drop=True)
result=pd.concat([result,toAdd], axis=1, sort=False)

result=result.drop(['Person','Feeling'],axis=1)"""

In [ ]:
""""count=0
for i in range(len(y_test)):
    print(y_test.reset_index(drop=True)[i],end= "-")
    print(LR.predict(X_test)[i], end='')
    if LR.predict(X_test)[i]==y_test.reset_index(drop=True)[i]:
        print(' well done! \n')
        count+=1
    else:
        print('\n')
print(count/len(y_test))""""

In [ ]:
"""result.head()"""

In [ ]:
"""#Create a new matrix 26x123x2 in which we have
#26 lines--->the number of records
#123 cols--->the pitch&Db recorded by SJapp
#2 elements for each cell---> Db, Hz
newVec=[]#gonna be our final result
for i in range(len(result)):
    temp=[]#gonna have 26 of this
    for k in range(len(result["Db"][i])):#fill our temp with a 123x2 matrix
        d=result["Db"][i][k]
        h=result["Hz"][i][k]
        temp.append([d,h])#for each cycle it writes a cell with 2 values (db,hz)
    newVec.append(temp)
newVec"""

In [ ]:
"""#trying to do with Hz and Db toghether
kmeans = KMeans(n_clusters=2, random_state=0).fit(newVec)"""

In [ ]:
"""count=0
for i in range(len(result)):
    pred=kmeans.predict(list([result["Hz"][i]+result["Db"][i]]))
    true=result["Male"][i]
    
    print(str(pred)+" "+str(true))
    if pred==true:
        count+=1
print(str(count)+"/"+str(len(result)))"""      

In [ ]:
"""###change the values 0 in pitch with the mean 
#NOT NECESSARY! DON'T RUN IT
for i in range(len(result)):
    for k in range(len(result["Hz"][i])):
        if result["Hz"][i][k]==0:
            result["Hz"][i][k]=mean(result["Hz"][i])"""